In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
df_tis = pd.read_csv(r"/content/drive/MyDrive/miRNA/tissue.csv", encoding = "ISO-8859-1")
df_cir = pd.read_csv(r"/content/drive/MyDrive/miRNA/circulationraw.csv", encoding = "ISO-8859-1")
df_gene = pd.read_csv(r"/content/drive/MyDrive/miRNA/genetics.csv", encoding = "ISO-8859-1")
df_all = df_tis.append(df_cir)
df_all = df_all.append(df_gene)
df_all

,category,mir,disease,pmid,description
0,tissue_expression_down,hsa-mir-125a,Breast Neoplasms,16103053,downregulation
1,tissue_expression_down,hsa-mir-125b-1,Breast Neoplasms,16103053,downregulation
2,tissue_expression_down,hsa-mir-125b-2,Breast Neoplasms,16103053,downregulation
3,tissue_expression_down,hsa-mir-145,Breast Neoplasms,16103053,downregulation
4,tissue_expression_down,hsa-mir-125a,"Carcinoma, Hepatocellular",16331254,downregulated
...,...,...,...,...,...
2360,genetics_overexpression_suppress,hsa-mir-27b,Atrial Fibrillation,29671258,Atrial overexpression of microRNA-27b attenuat...
2361,genetics_overexpression_suppress,hsa-mir-132,Heart Failure,29682535,Overexpression of microRNA-132 dramatically in...
2362,genetics_overexpression_suppress,hsa-mir-140,Intervertebral Disc Degeneration,29901170,Overexpression of miR-140-5p inhibits lipopoly...
2363,genetics_overexpression_suppress,hsa-mir-7,"Carcinoma, Hepatocellular",29936764,Overexpression of miR-7 can inhibit the prolif...


In [ ]:
df_test = df_all[(df_all['pmid'] == 19748927)]
df_test

,category,mir,disease,pmid,description
421,tissue_expression_down,hsa-mir-1,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."
422,tissue_expression_down,hsa-mir-143,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."
6408,tissue_expression_up,hsa-mir-130a,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."
6409,tissue_expression_up,hsa-mir-146b,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."
6410,tissue_expression_up,hsa-mir-21,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."
6411,tissue_expression_up,hsa-mir-31,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."
6412,tissue_expression_up,hsa-mir-377,Lung Neoplasms,19748927,"miR-21, mir-31, miR-130a, miR-146b and miR-377..."


In [ ]:
mirna_list = []
for i in tqdm(range(len(df_all))):
  if df_all.iloc[i]['mir'] not in mirna_list: mirna_list.append(df_all.iloc[i]['mir'])
len(mirna_list)

100%|██████████| 16039/16039 [00:03<00:00, 4498.76it/s]


1020

In [ ]:
# "hsa-mir-3" in mirna_list

In [ ]:
from pandas.core.frame import DataFrame
hairpin_name = []

with open("/content/drive/MyDrive/miRNA/hairpin.fa") as file:
  for line in file:
    # print(line)
    if ">" in line:
      hairpin_name.append(line.rstrip())
def mirToAcc(mir):
  search = mir
  acc = ""
  err = 0
  for i in range(len(hairpin_name)):
    target = hairpin_name[i]
    target = target.split(" ")
    target_mir = target[0][1:]
    if search.lower() == target_mir.lower():
      # print(hairpin_name[i])
      acc = target[1]
      err += 1
  if err > 1: raise BaseException("Find two index.")
  # if acc == "": print(mir)
  return acc

acc = [mirToAcc(mirna_list[i]) for i in (range(len(mirna_list)))]
dict_mir = {'mir': mirna_list, 'acc': acc}
df_trimed = DataFrame(dict_mir)
df_trimed

,mir,acc
0,hsa-mir-125a,MI0000469
1,hsa-mir-125b-1,MI0000446
2,hsa-mir-125b-2,MI0000470
3,hsa-mir-145,MI0000461
4,hsa-mir-126,MI0000471
...,...,...
1015,hsa-mir-544,
1016,hsa-mir-674,
1017,hsa-mir-156a,
1018,hsa-mir-548an,MI0016907


In [ ]:
df_trimed[df_trimed['mir'] == 'hsa-mir-3']

,mir,acc
779,hsa-mir-3,


In [ ]:
def getMatureName(site):
  response = requests.get(site)
  soup = BeautifulSoup(response.text, "html.parser")
  h2s = soup.find_all('h2')
  MatureName = []
  for h in h2s:
    h = str(h)
    if "Mature sequence" in h:
      MatureName.append(h[6:-6].replace("Mature sequence ",""))
  return MatureName
df_all_acc = df_trimed.drop(df_trimed[(df_trimed['acc'] == "")].index)
df_all_acc.reset_index(drop=True, inplace=True)

is_created = False


for i in tqdm(range(len(df_all_acc))):
# for i in tqdm(range(3)):

  site = r"https://www.mirbase.org/cgi-bin/mirna_entry.pl?acc=" + df_all_acc.iloc[i]["acc"]
  MatureNames = getMatureName(site)
  for MatureName in MatureNames:

    temp = df_all_acc.iloc[i]
    temp["MatureName"] = MatureName

    d = {'mir': [temp['mir']],
        'acc': [temp['acc']],
        'MatureName': [temp['MatureName']]}

    if is_created == False:
      df_MatureName = pd.DataFrame(data=d)
      is_created = True
    else:
      df_temp = pd.DataFrame(data=d)
      df_MatureName = df_MatureName.append(df_temp, ignore_index = True)
df_MatureName

100%|██████████| 794/794 [20:30<00:00,  1.55s/it]


,mir,acc,MatureName
0,hsa-mir-125a,MI0000469,hsa-miR-125a-5p
1,hsa-mir-125a,MI0000469,hsa-miR-125a-3p
2,hsa-mir-125b-1,MI0000446,hsa-miR-125b-5p
3,hsa-mir-125b-1,MI0000446,hsa-miR-125b-1-3p
4,hsa-mir-125b-2,MI0000470,hsa-miR-125b-5p
...,...,...,...
1282,hsa-mir-937,MI0005759,hsa-miR-937-5p
1283,hsa-mir-937,MI0005759,hsa-miR-937-3p
1284,hsa-mir-548an,MI0016907,hsa-miR-548an
1285,hsa-mir-541,MI0005539,hsa-miR-541-5p


In [ ]:
def getMIMAT(mirName):
  site = r"https://www.mirbase.org/cgi-bin/textsearch_json.pl?q=" + mirName
  new_query = mirName + '-'

  count = 0 
  accessions = []
  response = requests.get(site)
  soup = BeautifulSoup(response.text, "html.parser")
  # print(site)
  # print(response.json()["data"])
  # if response.json()["data"] == []: return ['No result'] 
  for i in range(len(response.json()["data"])):
    if "MIMAT" in response.json()["data"][i]["accession"]: accessions.append(response.json()["data"][i]["accession"])
    count += 1
  if count > 5:
    accessions = []
    for i in range(len(response.json()["data"])):
      # print(response.json()["data"][i]["id"].lower(), response.json()["data"][i]["accession"])
      # print(new_query in response.json()["data"][i]["id"].lower())
      # print("MIMAT" in response.json()["data"][i]["accession"])
      if (mirName == response.json()["data"][i]["id"].lower() or new_query in response.json()["data"][i]["id"].lower() or (mirName + 'a-') in response.json()["data"][i]["id"].lower() or (mirName + 'b-') in response.json()["data"][i]["id"].lower() or (mirName + 'c-') in response.json()["data"][i]["id"].lower() or (mirName + 'd-') in response.json()["data"][i]["id"].lower() or (mirName + 'e-') in response.json()["data"][i]["id"].lower()) and "MIMAT" in response.json()["data"][i]["accession"]: 
        accessions.append(response.json()["data"][i]["accession"])
  # if accessions != []:
  return accessions
  # else:
  #   return ['too many']

In [ ]:
getMIMAT("hsa-mir-1")

['MIMAT0031892', 'MIMAT0000416']

In [ ]:
df_sMIMAT = df_trimed.drop(df_trimed[(df_trimed['acc'] != "")].index)
df_sMIMAT.reset_index(drop=True, inplace=True)
is_created = False
delete_list = []
for i in tqdm(range(len(df_sMIMAT))):
# for i in tqdm(range(10)):
  site = df_sMIMAT.iloc[i]["mir"]
  MIMATs = getMIMAT(site)
  # print(site)
  # print(MIMATs)
  if MIMATs != []:

    for MIMAT in MIMATs:

      temp = df_sMIMAT.iloc[i]
      temp["MIMAT"] = MIMAT

      d = {'mir': [temp['mir']],
        'acc': [temp['acc']],
        'MIMAT': [temp['MIMAT']]}
      
      if is_created == False:
        # print("aa")
        df_MIMAT = pd.DataFrame(data=d)
        is_created = True
      else:
        df_temp = pd.DataFrame(data=d)
        df_MIMAT = df_MIMAT.append(df_temp, ignore_index = True)
  else:
    delete_list.append(site)
df_MIMAT

100%|██████████| 226/226 [07:41<00:00,  2.04s/it]


,mir,acc,MIMAT
0,hsa-mir-1,,MIMAT0031892
1,hsa-mir-1,,MIMAT0000416
2,hsa-mir-133a,,MIMAT0000427
3,hsa-mir-133a,,MIMAT0026478
4,hsa-mir-125b,,MIMAT0004603
...,...,...,...
434,hsa-mir-548a,,MIMAT0003251
435,hsa-mir-548a,,MIMAT0032115
436,hsa-mir-548a,,MIMAT0004803
437,hsa-mir-544,,MIMAT0003164


In [ ]:
len(df_MIMAT[df_MIMAT['MIMAT'] == 'too many'])

0

In [ ]:
df_mir_seq_convert = pd.read_csv(r"/content/drive/MyDrive/miRNA/df_mir_seq_convert.csv")
df_mir_seq_convert

,MatureName,MIMAT,mature_seq
0,cel-let-7-5p,MIMAT0000001,UGAGGUAGUAGGUUGUAUAGUU
1,cel-let-7-3p,MIMAT0015091,CUAUGCAAUUUUCUACCUUACC
2,cel-lin-4-5p,MIMAT0000002,UCCCUGAGACCUCAAGUGUGA
3,cel-lin-4-3p,MIMAT0015092,ACACCUGGGCUCUCCGGGUACC
4,cel-miR-1-5p,MIMAT0020301,CAUACUUCCUUACAUGCCCAUA
...,...,...,...
48880,smc-miR-12461-3p,MIMAT0050488,AGUAAAUGAUCAGCAUCCUCCA
48881,gga-miR-1784b-5p,MIMAT0050489,UUCUGCUCCUAUUUAAGUCAAU
48882,gga-miR-1784b-3p,MIMAT0050490,UGAUUUCAAUAAGAGCAGAAUU
48883,mdo-miR-7385g-3p,MIMAT0050491,AGCAUAGAAUGUCAGAUCUAG


In [ ]:
seq = []
MatureName=[]
for i in range(len(df_MIMAT)):
  n = df_mir_seq_convert[(df_mir_seq_convert["MIMAT"] == df_MIMAT.iloc[i]['MIMAT'])]
  # print(n)
  if np.any(n):
    # print(n.values[0])
    seq.append(n['mature_seq'].values[0])
    MatureName.append(n['MatureName'].values[0])
  else:
    seq.append("")
    MatureName.append("")
df_MIMAT['seq'] = seq
df_MIMAT['MatureName'] = MatureName
df_MIMAT[(df_MIMAT['mir'] == "hsa-mir-1")]

,mir,acc,MIMAT,seq,MatureName
0,hsa-mir-1,,MIMAT0022941,GUGGGGCCAGGCGGUGG,hsa-miR-1227-5p
1,hsa-mir-1,,MIMAT0000431,CAGUGGUUUUACCCUAUGGUAG,hsa-miR-140-5p
2,hsa-mir-1,,MIMAT0026740,ACAUUUUCCAGCCCAUUCA,hsa-miR-1250-3p
3,hsa-mir-1,,MIMAT0005577,UCACCAGCCCUGUGUUCCCUAG,hsa-miR-1226-3p
4,hsa-mir-1,,MIMAT0005891,UUUAGAGACGGGGUCUUGCUCU,hsa-miR-1303
...,...,...,...,...,...
413,hsa-mir-1,,MIMAT0049027,CUUGGCACCAUUAAAAAGUACA,hsa-miR-12133
414,hsa-mir-1,,MIMAT0019951,UCAGAUGAUCUAAAGGCCUAUA,hsa-miR-1245b-3p
415,hsa-mir-1,,MIMAT0004490,AGUUUUGCAUAGUUGCACUACA,hsa-miR-19a-5p
416,hsa-mir-1,,MIMAT0004599,GGUGCAGUGCUGCAUCUCUGGU,hsa-miR-143-5p


In [ ]:
seq = []
MatureName=[]
for i in range(len(df_MIMAT)):
  n = df_mir_seq_convert[(df_mir_seq_convert["MIMAT"] == df_MIMAT.iloc[i]['MIMAT'])]
  # print(n)
  if np.any(n):
    # print(n.values[0])
    seq.append(n['mature_seq'].values[0])
    MatureName.append(n['MatureName'].values[0])
  else:
    seq.append("")
    MatureName.append("")
df_MIMAT['seq'] = seq
df_MIMAT['MatureName'] = MatureName
df_MIMAT = df_MIMAT.drop(df_MIMAT[(df_MIMAT['seq'] == "")].index)
df_MIMAT.reset_index(drop=True, inplace=True)
df_MIMAT

In [ ]:
df_MIMAT

In [ ]:
df_MIMAT.iloc[436]['acc'] == None

True

In [ ]:
acc = []
def getMI(mirName):
  site = r"https://www.mirbase.org/cgi-bin/textsearch_json.pl?q=" + mirName
  count = 0 
  accessions = []
  response = requests.get(site)
  soup = BeautifulSoup(response.text, "html.parser")
  # print(site)
  # print(response.json()["data"])
  # if response.json()["data"] == []: return ['No result'] 
  for i in range(len(response.json()["data"])):
    if "MIMAT" not in response.json()["data"][i]["accession"] and "MI" in response.json()["data"][i]["accession"] and response.json()["data"][i]["deadMatch"] == 0:
      if (mirName + '-1') in response.json()["data"][i]["id"]:
        return response.json()["data"][i]["accession"]
      if (mirName + 'a') in response.json()["data"][i]["id"]:
        return response.json()["data"][i]["accession"]


  return "delete"



for i in tqdm(range(len(df_MIMAT))):
  mir_name = df_MIMAT.iloc[i]['mir']
  MI = getMI(mir_name)
  acc.append(MI)
df_MIMAT['acc'] = acc
df_MIMAT

100%|██████████| 428/428 [13:37<00:00,  1.91s/it]


,mir,acc,MIMAT,seq,MatureName
0,hsa-mir-1,MI0000651,MIMAT0031892,ACAUACUUCUUUAUAUGCCCAU,hsa-miR-1-5p
1,hsa-mir-1,MI0000651,MIMAT0000416,UGGAAUGUAAAGAAGUAUGUAU,hsa-miR-1-3p
2,hsa-mir-133a,MI0000450,MIMAT0000427,UUUGGUCCCCUUCAACCAGCUG,hsa-miR-133a-3p
3,hsa-mir-133a,MI0000450,MIMAT0026478,AGCUGGUAAAAUGGAACCAAAU,hsa-miR-133a-5p
4,hsa-mir-125b,MI0000446,MIMAT0004603,UCACAAGUCAGGCUCUUGGGAC,hsa-miR-125b-2-3p
...,...,...,...,...,...
432,hsa-mir-548a,MI0016690,MIMAT0003251,CAAAACUGGCAAUUACUUUUGC,hsa-miR-548a-3p
433,hsa-mir-548a,MI0016690,MIMAT0032115,AAAAGUAAUUGUGGUUUUUG,hsa-miR-548ae-5p
434,hsa-mir-548a,MI0016690,MIMAT0004803,AAAAGUAAUUGCGAGUUUUACC,hsa-miR-548a-5p
435,hsa-mir-544,MI0003515,MIMAT0003164,AUUCUGCAUUUUUAGCAAGUUC,hsa-miR-544a


In [ ]:
count = 0
for i in tqdm(range(len(df_MIMAT))):
  if df_MIMAT.iloc[i]['acc'] == None: print(df_MIMAT.iloc[i]['mir'])
count

100%|██████████| 428/428 [00:00<00:00, 11763.61it/s]


0

In [ ]:
df_MIMAT.drop(df_MIMAT[df_MIMAT['acc'] == 'delete'].index, inplace = True)

In [ ]:
seq = []
MIMAT = []
for i in range(len(df_MatureName)):
  n = df_mir_seq_convert[(df_mir_seq_convert["MatureName"] == df_MatureName.iloc[i]['MatureName'])]
  # print(n)
  if np.any(n):
    # print(n.values[0])
    seq.append(n['mature_seq'].values[0])
    MIMAT.append(n['MIMAT'].values[0])
  else:
    seq.append("")
    MIMAT.append("")
df_MatureName['seq'] = seq
df_MatureName['MIMAT'] = MIMAT
df_MatureName = df_MatureName.drop(df_MatureName[(df_MatureName['seq'] == "")].index)
df_MatureName.reset_index(drop=True, inplace=True)
df_MatureName

,mir,acc,MatureName,seq,MIMAT
0,hsa-mir-125a,MI0000469,hsa-miR-125a-5p,UCCCUGAGACCCUUUAACCUGUGA,MIMAT0000443
1,hsa-mir-125a,MI0000469,hsa-miR-125a-3p,ACAGGUGAGGUUCUUGGGAGCC,MIMAT0004602
2,hsa-mir-125b-1,MI0000446,hsa-miR-125b-5p,UCCCUGAGACCCUAACUUGUGA,MIMAT0000423
3,hsa-mir-125b-1,MI0000446,hsa-miR-125b-1-3p,ACGGGUUAGGCUCUUGGGAGCU,MIMAT0004592
4,hsa-mir-125b-2,MI0000470,hsa-miR-125b-5p,UCCCUGAGACCCUAACUUGUGA,MIMAT0000423
...,...,...,...,...,...
1282,hsa-mir-937,MI0005759,hsa-miR-937-5p,GUGAGUCAGGGUGGGGCUGG,MIMAT0022938
1283,hsa-mir-937,MI0005759,hsa-miR-937-3p,AUCCGCGCUCUGACUCUCUGCC,MIMAT0004980
1284,hsa-mir-548an,MI0016907,hsa-miR-548an,AAAAGGCAUUGUGGUUUUUG,MIMAT0019079
1285,hsa-mir-541,MI0005539,hsa-miR-541-5p,AAAGGAUUCUGCUGUCGGUCCCACU,MIMAT0004919


In [ ]:
df_convert = df_MIMAT.append(df_MatureName)
df_convert.reset_index(drop=True, inplace=True)
df_convert

,mir,acc,MIMAT,seq,MatureName
0,hsa-mir-1,MI0000651,MIMAT0031892,ACAUACUUCUUUAUAUGCCCAU,hsa-miR-1-5p
1,hsa-mir-1,MI0000651,MIMAT0000416,UGGAAUGUAAAGAAGUAUGUAU,hsa-miR-1-3p
2,hsa-mir-133a,MI0000450,MIMAT0000427,UUUGGUCCCCUUCAACCAGCUG,hsa-miR-133a-3p
3,hsa-mir-133a,MI0000450,MIMAT0026478,AGCUGGUAAAAUGGAACCAAAU,hsa-miR-133a-5p
4,hsa-mir-125b,MI0000446,MIMAT0004603,UCACAAGUCAGGCUCUUGGGAC,hsa-miR-125b-2-3p
...,...,...,...,...,...
1710,hsa-mir-937,MI0005759,MIMAT0022938,GUGAGUCAGGGUGGGGCUGG,hsa-miR-937-5p
1711,hsa-mir-937,MI0005759,MIMAT0004980,AUCCGCGCUCUGACUCUCUGCC,hsa-miR-937-3p
1712,hsa-mir-548an,MI0016907,MIMAT0019079,AAAAGGCAUUGUGGUUUUUG,hsa-miR-548an
1713,hsa-mir-541,MI0005539,MIMAT0004919,AAAGGAUUCUGCUGUCGGUCCCACU,hsa-miR-541-5p


In [ ]:
hairpin = open("/content/drive/MyDrive/miRNA/hairpin.fa")
i = 2
mir_name = []
mature_seq = []
MIMAT = []
line = hairpin.readline()
first = True
while line!="":
  i += 1
  if line[0] == '>':
    if first:
      first = False
      temp = ""
    else:
      mature_seq.append(temp)
      temp = ""
    line = line.rstrip()
    line = line.split(" ")
    MIMAT.append(line[1])
    mir_name.append(line[0][1:])
  else:
    temp += line.rstrip()

  line = hairpin.readline()
d = {'mirName': mir_name[:-1], "MI": MIMAT[:-1], 'seq': mature_seq}
df_hairpin_seq_convert = pd.DataFrame(data=d)
df_hairpin_seq_convert

,mirName,MI,seq
0,cel-let-7,MI0000001,UACACUGUGGAUCCGGUGAGGUAGUAGGUUGUAUAGUUUGGAAUAU...
1,cel-lin-4,MI0000002,AUGCUUCCGGCCUGUUCCCUGAGACCUCAAGUGUGAGUGUACUAUU...
2,cel-mir-1,MI0000003,AAAGUGACCGUACCGAGCUGCAUACUUCCUUACAUGCCCAUACUAU...
3,cel-mir-2,MI0000004,UAAACAGUAUACAGAAAGCCAUCAAAGCGGUGGUUGAUGUGUUGCA...
4,cel-mir-34,MI0000005,CGGACAAUGCUCGAGAGGCAGUGUGGUUAGCUGGUUGCAUAUUUCC...
...,...,...,...
38583,smc-mir-12460,MI0041069,UCAGUGGCAGAAGCUGGGAACUCUGAUUUUAAUUCGUUUUAAUUUA...
38584,smc-mir-12461,MI0041070,GAGGAUGCUGAUCAUUCACUGGCCCCCUGUGGACACGUGUGUUGCA...
38585,hsa-mir-9902-2,MI0041071,GCAGGGAAAGGGAACCCAGAAAUCUGGUAUGCCAGCAAAGAGAGUA...
38586,gga-mir-1784b,MI0041072,UUCUGCUCCUAUUUAAGUCAAUGGCAGAACUCUCACUGAUUUCAAU...


In [ ]:
hair_dict = {}
for i in range(len(df_hairpin_seq_convert)):
  MI = df_hairpin_seq_convert.iloc[i]["MI"]
  seq = df_hairpin_seq_convert.iloc[i]["seq"]
  hair_dict[MI] = seq

In [ ]:
df_convert[df_convert['acc'] == "MI0000278"]

,mir,acc,MIMAT,seq,MatureName
145,hsa-mir-196,MI0000278,MIMAT0001080,UAGGUAGUUUCCUGUUGUUGGG,hsa-miR-196b-5p
146,hsa-mir-196,MI0000278,MIMAT0037307,CAACAACAUUAAACCACCCGA,hsa-miR-196a-1-3p
147,hsa-mir-196,MI0000278,MIMAT0004562,CGGCAACAAGAAACUGCCUGAG,hsa-miR-196a-3p
148,hsa-mir-196,MI0000278,MIMAT0009201,UCGACAGCACGACACUGCCUUC,hsa-miR-196b-3p
149,hsa-mir-196,MI0000278,MIMAT0000226,UAGGUAGUUUCAUGUUGUUGGG,hsa-miR-196a-5p


In [ ]:
seqs = []
for i in range(len(df_convert)):
  MI = df_convert.iloc[i]["acc"]
  seqs.append(hair_dict[MI])
df_convert["H_seq"] = seqs
df_convert

,mir,acc,MIMAT,seq,MatureName,H_seq
0,hsa-mir-1,MI0000651,MIMAT0031892,ACAUACUUCUUUAUAUGCCCAU,hsa-miR-1-5p,UGGGAAACAUACUUCUUUAUAUGCCCAUAUGGACCUGCUAAGCUAU...
1,hsa-mir-1,MI0000651,MIMAT0000416,UGGAAUGUAAAGAAGUAUGUAU,hsa-miR-1-3p,UGGGAAACAUACUUCUUUAUAUGCCCAUAUGGACCUGCUAAGCUAU...
2,hsa-mir-133a,MI0000450,MIMAT0000427,UUUGGUCCCCUUCAACCAGCUG,hsa-miR-133a-3p,ACAAUGCUUUGCUAGAGCUGGUAAAAUGGAACCAAAUCGCCUCUUC...
3,hsa-mir-133a,MI0000450,MIMAT0026478,AGCUGGUAAAAUGGAACCAAAU,hsa-miR-133a-5p,ACAAUGCUUUGCUAGAGCUGGUAAAAUGGAACCAAAUCGCCUCUUC...
4,hsa-mir-125b,MI0000446,MIMAT0004603,UCACAAGUCAGGCUCUUGGGAC,hsa-miR-125b-2-3p,UGCGCUCCUCUCAGUCCCUGAGACCCUAACUUGUGAUGUUUACCGU...
...,...,...,...,...,...,...
1282,hsa-mir-937,MI0005759,MIMAT0022938,GUGAGUCAGGGUGGGGCUGG,hsa-miR-937-5p,AGCACUGCCCCCGGUGAGUCAGGGUGGGGCUGGCCCCCUGCUUCGU...
1283,hsa-mir-937,MI0005759,MIMAT0004980,AUCCGCGCUCUGACUCUCUGCC,hsa-miR-937-3p,AGCACUGCCCCCGGUGAGUCAGGGUGGGGCUGGCCCCCUGCUUCGU...
1284,hsa-mir-548an,MI0016907,MIMAT0019079,AAAAGGCAUUGUGGUUUUUG,hsa-miR-548an,CAUUAGGUUGGUGCAAAAGGCAUUGUGGUUUUUGCCUAUAAAAGUA...
1285,hsa-mir-541,MI0005539,MIMAT0004919,AAAGGAUUCUGCUGUCGGUCCCACU,hsa-miR-541-5p,ACGUCAGGGAAAGGAUUCUGCUGUCGGUCCCACUCCAAAGUUCACA...


In [ ]:

df_convert.to_csv(r"/content/drive/MyDrive/miRNA/pre2mature_pre.csv")

# Data Summary mir2miR

In [ ]:
df_tis = pd.read_csv(r"/content/drive/MyDrive/miRNA/tissue.csv", encoding = "ISO-8859-1")
df_cir = pd.read_csv(r"/content/drive/MyDrive/miRNA/circulationraw.csv", encoding = "ISO-8859-1")
df_gene = pd.read_csv(r"/content/drive/MyDrive/miRNA/genetics.csv", encoding = "ISO-8859-1")
df_all = df_tis.append(df_cir)
df_all = df_all.append(df_gene)
df_all

,category,mir,disease,pmid,description
0,tissue_expression_down,hsa-mir-125a,Breast Neoplasms,16103053,downregulation
1,tissue_expression_down,hsa-mir-125b-1,Breast Neoplasms,16103053,downregulation
2,tissue_expression_down,hsa-mir-125b-2,Breast Neoplasms,16103053,downregulation
3,tissue_expression_down,hsa-mir-145,Breast Neoplasms,16103053,downregulation
4,tissue_expression_down,hsa-mir-125a,"Carcinoma, Hepatocellular",16331254,downregulated
...,...,...,...,...,...
2360,genetics_overexpression_suppress,hsa-mir-27b,Atrial Fibrillation,29671258,Atrial overexpression of microRNA-27b attenuat...
2361,genetics_overexpression_suppress,hsa-mir-132,Heart Failure,29682535,Overexpression of microRNA-132 dramatically in...
2362,genetics_overexpression_suppress,hsa-mir-140,Intervertebral Disc Degeneration,29901170,Overexpression of miR-140-5p inhibits lipopoly...
2363,genetics_overexpression_suppress,hsa-mir-7,"Carcinoma, Hepatocellular",29936764,Overexpression of miR-7 can inhibit the prolif...


In [ ]:
df_convert = pd.read_csv(r"/content/drive/MyDrive/miRNA/pre2mature.csv", encoding = "ISO-8859-1")
df_convert.drop(columns=['Unnamed: 0', 'acc', 'MIMAT'])
df_convert

,Unnamed: 0,mir,acc,MIMAT,seq,MatureName
0,0,hsa-mir-1,NaN,MIMAT0022941,GUGGGGCCAGGCGGUGG,hsa-miR-1227-5p
1,1,hsa-mir-1,NaN,MIMAT0000431,CAGUGGUUUUACCCUAUGGUAG,hsa-miR-140-5p
2,2,hsa-mir-1,NaN,MIMAT0026740,ACAUUUUCCAGCCCAUUCA,hsa-miR-1250-3p
3,3,hsa-mir-1,NaN,MIMAT0005577,UCACCAGCCCUGUGUUCCCUAG,hsa-miR-1226-3p
4,4,hsa-mir-1,NaN,MIMAT0005891,UUUAGAGACGGGGUCUUGCUCU,hsa-miR-1303
...,...,...,...,...,...,...
3338,1282,hsa-mir-937,MI0005759,MIMAT0022938,GUGAGUCAGGGUGGGGCUGG,hsa-miR-937-5p
3339,1283,hsa-mir-937,MI0005759,MIMAT0004980,AUCCGCGCUCUGACUCUCUGCC,hsa-miR-937-3p
3340,1284,hsa-mir-548an,MI0016907,MIMAT0019079,AAAAGGCAUUGUGGUUUUUG,hsa-miR-548an
3341,1285,hsa-mir-541,MI0005539,MIMAT0004919,AAAGGAUUCUGCUGUCGGUCCCACU,hsa-miR-541-5p


In [ ]:
df_convert = df_convert.drop(columns=['Unnamed: 0', 'acc', 'MIMAT'])
df_convert

,mir,seq,MatureName
0,hsa-mir-1,GUGGGGCCAGGCGGUGG,hsa-miR-1227-5p
1,hsa-mir-1,CAGUGGUUUUACCCUAUGGUAG,hsa-miR-140-5p
2,hsa-mir-1,ACAUUUUCCAGCCCAUUCA,hsa-miR-1250-3p
3,hsa-mir-1,UCACCAGCCCUGUGUUCCCUAG,hsa-miR-1226-3p
4,hsa-mir-1,UUUAGAGACGGGGUCUUGCUCU,hsa-miR-1303
...,...,...,...
3338,hsa-mir-937,GUGAGUCAGGGUGGGGCUGG,hsa-miR-937-5p
3339,hsa-mir-937,AUCCGCGCUCUGACUCUCUGCC,hsa-miR-937-3p
3340,hsa-mir-548an,AAAAGGCAUUGUGGUUUUUG,hsa-miR-548an
3341,hsa-mir-541,AAAGGAUUCUGCUGUCGGUCCCACU,hsa-miR-541-5p


In [ ]:
freq = df_convert.groupby(['mir']).count()
freq.sort_values(by=['seq'], ascending=False).head(30)

,seq,MatureName
mir,,
hsa-mir-3,420,420
hsa-mir-1,406,406
hsa-mir-12,153,153
hsa-mir-7,94,94
hsa-mir-19,51,51
hsa-mir-9,51,51
hsa-mir-10,49,49
hsa-mir-548a,42,42
hsa-mir-30,36,36


In [ ]:
df_convert[(df_convert['mir'] == 'hsa-mir-1')]

,mir,acc,MIMAT,seq,MatureName
0,hsa-mir-1,,MIMAT0022941,GUGGGGCCAGGCGGUGG,hsa-miR-1227-5p
1,hsa-mir-1,,MIMAT0000431,CAGUGGUUUUACCCUAUGGUAG,hsa-miR-140-5p
2,hsa-mir-1,,MIMAT0026740,ACAUUUUCCAGCCCAUUCA,hsa-miR-1250-3p
3,hsa-mir-1,,MIMAT0005577,UCACCAGCCCUGUGUUCCCUAG,hsa-miR-1226-3p
4,hsa-mir-1,,MIMAT0005891,UUUAGAGACGGGGUCUUGCUCU,hsa-miR-1303
...,...,...,...,...,...
401,hsa-mir-1,,MIMAT0049027,CUUGGCACCAUUAAAAAGUACA,hsa-miR-12133
402,hsa-mir-1,,MIMAT0019951,UCAGAUGAUCUAAAGGCCUAUA,hsa-miR-1245b-3p
403,hsa-mir-1,,MIMAT0004490,AGUUUUGCAUAGUUGCACUACA,hsa-miR-19a-5p
404,hsa-mir-1,,MIMAT0004599,GGUGCAGUGCUGCAUCUCUGGU,hsa-miR-143-5p


In [ ]:
df_1 = df_convert[(df_convert['mir'] == 'hsa-mir-1')]
df_1[df_1['MatureName'].str.contains("hsa-miR-1-")]
# df_1[df_1['MatureName'] == ("hsa-miR-1")]

,mir,seq,MatureName
51,hsa-mir-1,ACAUACUUCUUUAUAUGCCCAU,hsa-miR-1-5p
177,hsa-mir-1,UGGAAUGUAAAGAAGUAUGUAU,hsa-miR-1-3p


In [ ]:
df_3 = df_convert[(df_convert['mir'] == 'hsa-mir-12')]
df_3
df_3[df_3['MatureName'].str.contains("hsa-miR-12-")]

,mir,seq,MatureName


In [ ]:
df_all[df_all['mir'] == 'hsa-mir-12']

,category,mir,disease,pmid,description
2999,circulation_biomarker_diagnosis_ns,hsa-mir-12,Viral Infectious Disease,30146762,"To date, numerous biomarkers have been identif..."


In [ ]:
df_all[df_all['pmid'] == 30146762 ]

,category,mir,disease,pmid,description
2999,circulation_biomarker_diagnosis_ns,hsa-mir-12,Viral Infectious Disease,30146762,"To date, numerous biomarkers have been identif..."
3000,circulation_biomarker_diagnosis_ns,hsa-mir-125,Hepatitis [unspecific],30146762,"To date, numerous biomarkers have been identif..."
3001,circulation_biomarker_diagnosis_ns,hsa-mir-1260,Influenza,30146762,"To date, numerous biomarkers have been identif..."
3002,circulation_biomarker_diagnosis_ns,hsa-mir-155,Influenza,30146762,"To date, numerous biomarkers have been identif..."
3003,circulation_biomarker_diagnosis_ns,hsa-mir-15a,Sepsis,30146762,"To date, numerous biomarkers have been identif..."
3004,circulation_biomarker_diagnosis_ns,hsa-mir-182,Sepsis,30146762,"To date, numerous biomarkers have been identif..."
3005,circulation_biomarker_diagnosis_ns,hsa-mir-184,Viral Infectious Disease,30146762,"To date, numerous biomarkers have been identif..."
3006,circulation_biomarker_diagnosis_ns,hsa-mir-29b,Hepatitis [unspecific],30146762,"To date, numerous biomarkers have been identif..."
3007,circulation_biomarker_diagnosis_ns,hsa-mir-320,Inflammatory Bowel Diseases,30146762,"To date, numerous biomarkers have been identif..."
3008,circulation_biomarker_diagnosis_ns,hsa-mir-486,Sepsis,30146762,"To date, numerous biomarkers have been identif..."


# Data Summary epi and GWAS


In [ ]:
df_epi = pd.read_csv(r"/content/drive/MyDrive/miRNA/epigenetics.csv", encoding = "ISO-8859-1")
df_epi

,category,mir,disease,pmid,description
0,epigenetics,hsa-mir-195,"Carcinoma, Hepatocellular",17188425,miRNA-195 expression is decreased in HCC compa...
1,epigenetics,hsa-mir-127,Colon Neoplasms,17355635,Human miR-127 is embedded in a CpG island and ...
2,epigenetics,hsa-mir-127,Prostate Neoplasms,17355635,Human miR-127 is embedded in a CpG island and ...
3,epigenetics,hsa-mir-127,Urinary Bladder Cancer,17355635,Human miR-127 is embedded in a CpG island and ...
4,epigenetics,hsa-mir-29a,Lung Neoplasms,17890317,"mir-29 family (29a,b,c) reverts aberrant methy..."
...,...,...,...,...,...
666,epigenetics,hsa-mir-107,Ovarian Neoplasms,30363055,"Hypermethylation of miR-107, miR-130b, miR-203..."
667,epigenetics,hsa-mir-1258,Ovarian Neoplasms,30363055,"Hypermethylation of miR-107, miR-130b, miR-203..."
668,epigenetics,hsa-mir-130b,Ovarian Neoplasms,30363055,"Hypermethylation of miR-107, miR-130b, miR-203..."
669,epigenetics,hsa-mir-203a,Ovarian Neoplasms,30363055,"Hypermethylation of miR-107, miR-130b, miR-203..."


In [ ]:
df_GWAS = df_all[(df_all['category'] == 'genetics_GWAS')]
df_GWAS

,category,mir,disease,pmid,description
0,genetics_GWAS,hsa-mir-142,Leukemia,15737576,Some human miRNAs are linked to leukemias: the...
1,genetics_GWAS,hsa-mir-189,Tourette Syndrome,16224024,we identified a frameshift mutation and two in...
2,genetics_GWAS,hsa-mir-16-1,"Leukemia, Lymphocytic, Chronic, B-Cell",16251535,mutation
3,genetics_GWAS,hsa-mir-187,"Leukemia, Lymphocytic, Chronic, B-Cell",16251535,mutation
4,genetics_GWAS,hsa-mir-206,"Leukemia, Lymphocytic, Chronic, B-Cell",16251535,mutation
...,...,...,...,...,...
1075,genetics_GWAS,hsa-mir-423,Coronary Artery Disease,30289085,Potential impact of microRNA-423 gene variabil...
1076,genetics_GWAS,hsa-mir-125a,"Carcinoma, Lung",30302847,Regulation of miR-125a expression by rs1297644...
1077,genetics_GWAS,hsa-mir-589,Cataract,30315181,luciferase reporter gene expression can be dow...
1078,genetics_GWAS,hsa-mir-579,Anxiety Disorders,30341278,A functional genetic variation of SLC6A2 repre...


In [ ]:
df_epi[df_epi['description'].str.contains("p<")]

,category,mir,disease,pmid,description
153,epigenetics,hsa-mir-129-2,Esophageal Neoplasms,21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
154,epigenetics,hsa-mir-129-2,"Squamous Cell Carcinoma, Esophageal",21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
155,epigenetics,hsa-mir-34a,Esophageal Neoplasms,21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
156,epigenetics,hsa-mir-34a,"Squamous Cell Carcinoma, Esophageal",21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
157,epigenetics,hsa-mir-34b,Esophageal Neoplasms,21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
158,epigenetics,hsa-mir-34b,"Squamous Cell Carcinoma, Esophageal",21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
159,epigenetics,hsa-mir-34c,Esophageal Neoplasms,21547903,"The methylation ratio of miR-34a, miR-34b/c an..."
160,epigenetics,hsa-mir-34c,"Squamous Cell Carcinoma, Esophageal",21547903,"The methylation ratio of miR-34a, miR-34b/c an..."


In [ ]:
df_GWAS[df_GWAS['description'].str.contains("p<")]

,category,mir,disease,pmid,description
275,genetics_GWAS,hsa-mir-196a-2,"Carcinoma, Lung, Non-Small-Cell",21902575,The authors clearly detected a significant inc...
